In [36]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

In [37]:
filePath = "/Users/uditagupta93/Desktop/Udita/NYU Tandon/Data Science/Assignment2/train.csv"
def loadTrainFile(filePath):
    #read a csv using pandas
    return pd.read_csv(filePath)

df_train = loadTrainFile(filePath)

In [38]:
df_train.head(5)

,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,LEAVE
0,zero,31953,0,6,313378,161,0,4,unsat,little,no,1
1,one,36147,0,13,800586,244,0,6,unsat,little,considering,1
2,one,27273,230,0,305049,201,16,15,unsat,very_little,perhaps,1
3,zero,120070,38,33,788235,780,3,2,unsat,very_high,considering,0
4,one,29215,208,85,224784,241,21,1,very_unsat,little,never_thought,1


In [39]:
df_train.shape

(18000, 12)

In [40]:
def cleanFile(df):
    df.dropna(how='any',inplace=True)
    df = pd.get_dummies(df, columns=["COLLEGE", "REPORTED_SATISFACTION", "REPORTED_USAGE_LEVEL", "CONSIDERING_CHANGE_OF_PLAN"], prefix=["COLLEGE", "REPORTED_SATISFACTION", "REPORTED_USAGE_LEVEL", "CONSIDERING_CHANGE_OF_PLAN"])
    df_features = df.drop("LEAVE", 1)
    
    # Create x, where x the 'HOUSE' column's value as floats
    x1 = df_features[['HOUSE']].values.astype(float)
    x2 = df_features[['OVERAGE']].values.astype(float)
    x3 = df_features[['LEFTOVER']].values.astype(float)
    x4 = df_features[['HANDSET_PRICE']].values.astype(float)
    x5 = df_features[['OVER_15MINS_CALLS_PER_MONTH']].values.astype(float)
    x6 = df_features[['AVERAGE_CALL_DURATION']].values.astype(float)
    x7 = df_features[['INCOME']].values.astype(float)

    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler()

    # Create an object to transform the data to fit minmax processor
    x1_scaled = min_max_scaler.fit_transform(x1)
    x2_scaled = min_max_scaler.fit_transform(x2)
    x3_scaled = min_max_scaler.fit_transform(x3)
    x4_scaled = min_max_scaler.fit_transform(x4)
    x5_scaled = min_max_scaler.fit_transform(x5)
    x6_scaled = min_max_scaler.fit_transform(x6)
    x7_scaled = min_max_scaler.fit_transform(x7)

    # Run the normalizer on the dataframe
    df_normalized_HOUSE = pd.DataFrame(x1_scaled)
    df_normalized_OVERAGE = pd.DataFrame(x2_scaled)
    df_normalized_LEFTOVER = pd.DataFrame(x3_scaled)
    df_normalized_HANDSET_PRICE = pd.DataFrame(x4_scaled)
    df_normalized_OVER_15MINS_CALLS_PER_MONTH = pd.DataFrame(x5_scaled)
    df_normalized_AVERAGE_CALL_DURATION = pd.DataFrame(x6_scaled)
    df_normalized_INCOME = pd.DataFrame(x7_scaled)
    
    df_features["HOUSE"] = df_normalized_HOUSE
    df_features["OVERAGE"] = df_normalized_OVERAGE
    df_features["LEFTOVER"] = df_normalized_LEFTOVER
    df_features["HANDSET_PRICE"] = df_normalized_HANDSET_PRICE
    df_features["OVER_15MINS_CALLS_PER_MONTH"] = df_normalized_OVER_15MINS_CALLS_PER_MONTH
    df_features["AVERAGE_CALL_DURATION"] = df_normalized_AVERAGE_CALL_DURATION
    df_features["INCOME"] = df_normalized_INCOME
    
    return df,df_features

df_train_cleaned, df_train_cleaned_features = cleanFile(df_train)


In [41]:
df_train_cleaned_features.dtypes

INCOME                                                 float64
OVERAGE                                                float64
LEFTOVER                                               float64
HOUSE                                                  float64
HANDSET_PRICE                                          float64
OVER_15MINS_CALLS_PER_MONTH                            float64
AVERAGE_CALL_DURATION                                  float64
COLLEGE_one                                              uint8
COLLEGE_zero                                             uint8
REPORTED_SATISFACTION_avg                                uint8
REPORTED_SATISFACTION_sat                                uint8
REPORTED_SATISFACTION_unsat                              uint8
REPORTED_SATISFACTION_very_sat                           uint8
REPORTED_SATISFACTION_very_unsat                         uint8
REPORTED_USAGE_LEVEL_avg                                 uint8
REPORTED_USAGE_LEVEL_high                              

In [43]:
#selecting features
features = list(df_train_cleaned_features.columns[:])
print(features)
Y = df_train_cleaned["LEAVE"]
X = df_train_cleaned_features[features]
print(X.shape)

['INCOME', 'OVERAGE', 'LEFTOVER', 'HOUSE', 'HANDSET_PRICE', 'OVER_15MINS_CALLS_PER_MONTH', 'AVERAGE_CALL_DURATION', 'COLLEGE_one', 'COLLEGE_zero', 'REPORTED_SATISFACTION_avg', 'REPORTED_SATISFACTION_sat', 'REPORTED_SATISFACTION_unsat', 'REPORTED_SATISFACTION_very_sat', 'REPORTED_SATISFACTION_very_unsat', 'REPORTED_USAGE_LEVEL_avg', 'REPORTED_USAGE_LEVEL_high', 'REPORTED_USAGE_LEVEL_little', 'REPORTED_USAGE_LEVEL_very_high', 'REPORTED_USAGE_LEVEL_very_little', 'CONSIDERING_CHANGE_OF_PLAN_actively_looking_into_it', 'CONSIDERING_CHANGE_OF_PLAN_considering', 'CONSIDERING_CHANGE_OF_PLAN_never_thought', 'CONSIDERING_CHANGE_OF_PLAN_no', 'CONSIDERING_CHANGE_OF_PLAN_perhaps']
(18000, 24)


In [44]:
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.1, random_state = 253)

In [45]:
# Decision Trees
model_decTree = DecisionTreeClassifier(criterion = "entropy",
 max_depth=4, min_samples_leaf=145, random_state=253)
model_decTree.fit(X_train, y_train)

y_pred_dt = model_decTree.predict(X_test)
print(y_pred_dt)
print("Decision Tree Accuracy is: ", accuracy_score(y_test,y_pred_dt)*100)
#clf_entropy.feature_importances_


[0 1 0 ..., 1 0 0]
Decision Tree Accuracy is:  73.0


In [46]:
# SVM
#Import Library
from sklearn import svm
#Assumed you have, X (predictor) and Y (target) for training data set and x_test(predictor) of test_dataset
# Create SVM classification object 
model_svm = svm.SVC(kernel='poly', C=0.01, gamma=1, random_state=253) 
# there is various option associated with it, like changing kernel, gamma and C value. Will discuss more # about it in next section.Train the model using the training sets and check score
model_svm.fit(X_train, y_train)

#Predict Output
y_pred_svm =model_svm.predict(X_test)
print(y_pred_svm)
print("SVM Accuracy is ", accuracy_score(y_test,y_pred_svm)*100)

[0 0 1 ..., 1 0 1]
SVM Accuracy is  68.2222222222


In [47]:
#Logistic Regr
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(C = 0.5, random_state=253, penalty='l2')
model_lr.fit(X_train,y_train)

y_pred_lr = model_lr.predict(X_test)
print(y_pred_lr)
print("Logistic Regr Accuracy is ", accuracy_score(y_test,y_pred_lr)*100)

[0 0 1 ..., 1 0 1]
Logistic Regr Accuracy is  65.2222222222


In [48]:
sum_list = y_pred_dt + y_pred_lr + y_pred_svm
print(sum_list)

[0 1 2 ..., 3 0 2]


In [49]:
final_class_label = []
for i in sum_list:
    if(i > 1):
        final_class_label.append(1)
    else:
        final_class_label.append(0)
        
print(final_class_label)

[0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 

In [50]:
count=0
for i in range(len(y_test)):
    if(list(y_test)[i] == final_class_label[i]):
        count+=1
        
accuracy_score = (count/len(y_test))*100
print("Final Classifier Accuracy: ",accuracy_score)

Final Classifier Accuracy:  69.16666666666667


In [51]:
#Test Submission

In [52]:
filePath = "/Users/uditagupta93/Desktop/Udita/NYU Tandon/Data Science/Assignment2/test.csv"
df_test = loadTrainFile(filePath)
df_test.shape

(2000, 11)

In [53]:
def cleanTestFile(df):
    df.dropna(how='any',inplace=True)
    df = pd.get_dummies(df, columns=["COLLEGE", "REPORTED_SATISFACTION", "REPORTED_USAGE_LEVEL", "CONSIDERING_CHANGE_OF_PLAN"], prefix=["COLLEGE", "REPORTED_SATISFACTION", "REPORTED_USAGE_LEVEL", "CONSIDERING_CHANGE_OF_PLAN"])
    df_features = df
    
    # Create x, where x the 'HOUSE' column's value as floats
    x1 = df_features[['HOUSE']].values.astype(float)
    x2 = df_features[['OVERAGE']].values.astype(float)
    x3 = df_features[['LEFTOVER']].values.astype(float)
    x4 = df_features[['HANDSET_PRICE']].values.astype(float)
    x5 = df_features[['OVER_15MINS_CALLS_PER_MONTH']].values.astype(float)
    x6 = df_features[['AVERAGE_CALL_DURATION']].values.astype(float)
    x7 = df_features[['INCOME']].values.astype(float)

    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler()

    # Create an object to transform the data to fit minmax processor
    x1_scaled = min_max_scaler.fit_transform(x1)
    x2_scaled = min_max_scaler.fit_transform(x2)
    x3_scaled = min_max_scaler.fit_transform(x3)
    x4_scaled = min_max_scaler.fit_transform(x4)
    x5_scaled = min_max_scaler.fit_transform(x5)
    x6_scaled = min_max_scaler.fit_transform(x6)
    x7_scaled = min_max_scaler.fit_transform(x7)

    # Run the normalizer on the dataframe
    df_normalized_HOUSE = pd.DataFrame(x1_scaled)
    df_normalized_OVERAGE = pd.DataFrame(x2_scaled)
    df_normalized_LEFTOVER = pd.DataFrame(x3_scaled)
    df_normalized_HANDSET_PRICE = pd.DataFrame(x4_scaled)
    df_normalized_OVER_15MINS_CALLS_PER_MONTH = pd.DataFrame(x5_scaled)
    df_normalized_AVERAGE_CALL_DURATION = pd.DataFrame(x6_scaled)
    df_normalized_INCOME = pd.DataFrame(x7_scaled)
    
    df_features["HOUSE"] = df_normalized_HOUSE
    df_features["OVERAGE"] = df_normalized_OVERAGE
    df_features["LEFTOVER"] = df_normalized_LEFTOVER
    df_features["HANDSET_PRICE"] = df_normalized_HANDSET_PRICE
    df_features["OVER_15MINS_CALLS_PER_MONTH"] = df_normalized_OVER_15MINS_CALLS_PER_MONTH
    df_features["AVERAGE_CALL_DURATION"] = df_normalized_AVERAGE_CALL_DURATION
    df_features["INCOME"] = df_normalized_INCOME
    
    return df_features

df_test_cleaned_features = cleanTestFile(df_test)
#df_test_cleaned_features = df_test_cleaned_features.drop(['HANDSET_PRICE', 'OVER_15MINS_CALLS_PER_MONTH', 'AVERAGE_CALL_DURATION', 'COLLEGE_one', 'COLLEGE_zero', 'REPORTED_SATISFACTION_avg', 'REPORTED_SATISFACTION_sat', 'REPORTED_SATISFACTION_unsat', 'REPORTED_SATISFACTION_very_sat', 'REPORTED_SATISFACTION_very_unsat', 'REPORTED_USAGE_LEVEL_avg', 'REPORTED_USAGE_LEVEL_high', 'REPORTED_USAGE_LEVEL_little', 'REPORTED_USAGE_LEVEL_very_high', 'REPORTED_USAGE_LEVEL_very_little', 'CONSIDERING_CHANGE_OF_PLAN_actively_looking_into_it', 'CONSIDERING_CHANGE_OF_PLAN_considering', 'CONSIDERING_CHANGE_OF_PLAN_never_thought', 'CONSIDERING_CHANGE_OF_PLAN_no', 'CONSIDERING_CHANGE_OF_PLAN_perhaps'], axis =1)
df_test_cleaned_features.head(5)
#df_test_cleaned_features.shape

,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,COLLEGE_one,COLLEGE_zero,REPORTED_SATISFACTION_avg,...,REPORTED_USAGE_LEVEL_avg,REPORTED_USAGE_LEVEL_high,REPORTED_USAGE_LEVEL_little,REPORTED_USAGE_LEVEL_very_high,REPORTED_USAGE_LEVEL_very_little,CONSIDERING_CHANGE_OF_PLAN_actively_looking_into_it,CONSIDERING_CHANGE_OF_PLAN_considering,CONSIDERING_CHANGE_OF_PLAN_never_thought,CONSIDERING_CHANGE_OF_PLAN_no,CONSIDERING_CHANGE_OF_PLAN_perhaps
0,0.878291,0.696552,0.000000,0.133805,0.561769,1.000000,0.857143,1,0,0,...,0,0,0,0,1,1,0,0,0,0
1,0.153746,0.227586,0.505618,0.551604,0.078023,0.103448,0.071429,0,1,0,...,0,1,0,0,0,1,0,0,0,0
2,0.228058,0.658621,0.685393,0.983938,0.292588,0.724138,0.000000,0,1,1,...,0,0,0,0,1,0,0,0,1,0
3,0.090788,0.000000,0.000000,0.122965,0.222367,0.034483,0.642857,1,0,1,...,0,0,0,1,0,0,1,0,0,0
4,0.145733,0.000000,0.000000,0.113118,0.141743,0.896552,0.500000,0,1,0,...,0,0,1,0,0,0,0,0,0,1


In [54]:
y_pred_dt = model_decTree.predict(df_test_cleaned_features)
y_pred_svm = model_svm.predict(df_test_cleaned_features)
y_pred_lr = model_lr.predict(df_test_cleaned_features)

In [55]:
sum_list = y_pred_dt + y_pred_lr + y_pred_svm
print(sum_list)

[0 3 3 ..., 0 2 2]


In [56]:
final_class_label = []
for i in sum_list:
    if(i > 1):
        final_class_label.append(1)
    else:
        final_class_label.append(0)
        
print(final_class_label)

[0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 

In [57]:
df_test_cleaned_features["LEAVE"] = final_class_label

In [58]:
df_test_cleaned_features.head(5)

,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,COLLEGE_one,COLLEGE_zero,REPORTED_SATISFACTION_avg,...,REPORTED_USAGE_LEVEL_high,REPORTED_USAGE_LEVEL_little,REPORTED_USAGE_LEVEL_very_high,REPORTED_USAGE_LEVEL_very_little,CONSIDERING_CHANGE_OF_PLAN_actively_looking_into_it,CONSIDERING_CHANGE_OF_PLAN_considering,CONSIDERING_CHANGE_OF_PLAN_never_thought,CONSIDERING_CHANGE_OF_PLAN_no,CONSIDERING_CHANGE_OF_PLAN_perhaps,LEAVE
0,0.878291,0.696552,0.000000,0.133805,0.561769,1.000000,0.857143,1,0,0,...,0,0,0,1,1,0,0,0,0,0
1,0.153746,0.227586,0.505618,0.551604,0.078023,0.103448,0.071429,0,1,0,...,1,0,0,0,1,0,0,0,0,1
2,0.228058,0.658621,0.685393,0.983938,0.292588,0.724138,0.000000,0,1,1,...,0,0,0,1,0,0,0,1,0,1
3,0.090788,0.000000,0.000000,0.122965,0.222367,0.034483,0.642857,1,0,1,...,0,0,1,0,0,1,0,0,0,0
4,0.145733,0.000000,0.000000,0.113118,0.141743,0.896552,0.500000,0,1,0,...,0,1,0,0,0,0,0,0,1,0


In [59]:
df_test_cleaned_features["LEAVE"].to_csv("/Users/uditagupta93/Desktop/Udita/NYU Tandon/Data Science/Assignment2/test_upload8.csv")